This is a fork of [this](https://www.kaggle.com/code/jdoesv/topics-identification) notebook by [jdoesv](https://www.kaggle.com/jdoesv).

I have added some code to save the model and the BERTopic package so I can use it in an offline inference kernel. 

In [ ]:
import glob, pandas as pd, numpy as np, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm

In [ ]:
!pip install bertopic -qq --target=/kaggle/working/site-packages
import sys
sys.path.append('/kaggle/working/site-packages')

In [ ]:
from bertopic import BERTopic

In [ ]:
sws = stopwords.words("english") + ["n't",  "'s", "'ve"]

In [ ]:
fls = glob.glob("../input/feedback-prize-2021/train/*.txt")
docs = []
for fl in tqdm(fls):
    with open(fl) as f:
        txt = f.read()
        word_tokens = word_tokenize(txt)
        txt = " ".join([w for w in word_tokens if not w.lower() in sws])
    docs.append(txt)

In [ ]:
topic_model = BERTopic(n_gram_range=(1, 3), top_n_words=5, verbose=True)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
tm_meta = topic_model.get_topic_info()
tm_meta.to_csv("topic_model_metadata.csv", index=False)
display(tm_meta)

pred_topics = pd.DataFrame()
dids = list(map(lambda fl: fl.split("/")[-1].split(".")[0], fls))
pred_topics["id"] = dids
pred_topics["topic"] = topics
pred_topics['prob'] = probs
pred_topics.to_csv("topic_model_feedback.csv", index=False)
pred_topics

In [ ]:
topic_model.save("feedback_2021_topic_model")